<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/NLPAssignment2LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Best Trained Model with output is at the end of the file

#LSTM

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt


#The file iam importing has been cleaned . (Contains stopwords) and its lemmatized to reduce vocabulory size

In [ ]:
columns=pd.read_csv('/content/fulldatafullcleaned.csv',nrows=0).columns.tolist()
df=pd.read_csv('/content/fulldatafullcleaned.csv',usecols=[columns[1],columns[2]])
df.describe()
sentences_count_with_nan=len(df)
df.dropna(inplace=True)
sentences_count_without_nan=len(df)
print('Before Nan ',sentences_count_with_nan)
print('After Nan ',sentences_count_without_nan)

df.head()
df.describe()



Before Nan  29617
After Nan  29574


,SENTENCES,MEANING
count,29574,29574
unique,21650,22116
top,good,اچھی
freq,290,290


In [ ]:

df = df.drop_duplicates()
df.describe()

,SENTENCES,MEANING
count,22987,22987
unique,21650,22116
top,dont come here,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,8,35


In [ ]:
#90 sentences are being covered
threshold = 30
# Count sentences exceeding the threshold
count_exceeding = df[df['SENTENCES '].str.split().str.len() > threshold].shape[0]
print(count_exceeding,len(df))

2501 22987


In [ ]:
df = df[df['SENTENCES '].str.split().str.len() <= threshold]


In [ ]:
df.describe()

,SENTENCES,MEANING
count,20486,20486
unique,19170,19617
top,dont come here,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,8,35


In [ ]:
threshold = 30
# Count sentences exceeding the threshold
count_exceeding = df[df['MEANING'].str.split().str.len() > threshold].shape[0]
print(count_exceeding,len(df))

975 20486


In [ ]:
df = df[df['MEANING'].str.split().str.len() <= threshold]


In [ ]:


df.describe()

,SENTENCES,MEANING
count,19511,19511
unique,18222,18642
top,dont come here,(گوگل کے ذریعہ ترجمہ شدہ) (اصل)
freq,8,35


In [ ]:
df.head()

,SENTENCES,MEANING
0,how can i communicate with my parents,میں اپنے والدین سے کیسے بات کروں
1,how can i make friends,میں دوست کیسے بنائوں
2,why do i get so sad,میں اتنا اداس کیوں ہوں
3,if youve asked yourself such questions youre n...,اگر آپ نے اپنے آپ سے ایسے سوالات کیے ہیں تو آپ...
4,depending on where youve turned for guidance y...,اس بات پر منحصر ہے کہ آپ رہنمائی کے لیے کہاں گ...


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# Initialize Tokenizers
eng_tokenizer = Tokenizer(filters='')  # Disable default filters to keep special tokens
urd_tokenizer = Tokenizer(filters='')  # Disable default filters to keep special tokens

# Manually add the special tokens to the text before tokenization
df['SENTENCES '] = df['SENTENCES '].apply(lambda x: '<start> ' + x + ' <end>')
df['MEANING'] = df['MEANING'].apply(lambda x: '<start> ' + x + ' <end>')

# Now fit tokenizers on the respective texts
eng_tokenizer.fit_on_texts(df['SENTENCES '])
urd_tokenizer.fit_on_texts(df['MEANING'])

# Vocabulary sizes
eng_vocab_size = len(eng_tokenizer.word_index) + 1
urd_vocab_size = len(urd_tokenizer.word_index) + 1

print('English Vocabulary Size:', eng_vocab_size)
print('Urdu Vocabulary Size:', urd_vocab_size)

# Verify the presence of special tokens in the word_index
print("Start token index (Urdu):", urd_tokenizer.word_index['<start>'])
print("End token index (Urdu):", urd_tokenizer.word_index['<end>'])


English Vocabulary Size: 11219
Urdu Vocabulary Size: 12003
Start token index (Urdu): 1
End token index (Urdu): 2


In [ ]:
df.head()
print("Start token index (Urdu):", urd_tokenizer.word_index['<start>'])
print("End token index (Urdu):", urd_tokenizer.word_index['<end>'])


Start token index (Urdu): 1
End token index (Urdu): 2


In [ ]:
df.head()
eng_tokenizer.word_index['<start>']
eng_vocab_size = len(eng_tokenizer.word_index) + 1
urd_vocab_size = len(urd_tokenizer.word_index) + 1

#Encoder Approch

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Concatenate, Dense, TimeDistributed
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters
embedding_dim = 512  # Dimension of the embedding layer
max_length = 30  # Maximum length for input and output sequences

# Encoder
encoder_input = Input(shape=(max_length,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)(encoder_input)
encoder_lstm = Bidirectional(LSTM(256, return_state=True, return_sequences=True))
encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
encoder_lstm = Bidirectional(LSTM(128, return_state=True, return_sequences=True))
encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

encoder_output = Dropout(0.2)(encoder_output)


# Combine the states from the forward and backward LSTMs
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Decoder
decoder_input = Input(shape=(max_length,))
decoder_embedding = Embedding(input_dim=urd_vocab_size, output_dim=embedding_dim)(decoder_input)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
# decoder_lstm = LSTM(512, return_sequences=True, return_state=True)
# decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

decoder_output = BatchNormalization()(decoder_output)

decoder_output = Dropout(0.1)(decoder_output)
decoder_dense = TimeDistributed(Dense(urd_vocab_size, activation='softmax'))
decoder_output = decoder_dense(decoder_output)

# Define the model
model = Model([encoder_input, decoder_input], decoder_output)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
start_token_id = 1  # Assume <start> token is 1

# Initialize decoder input data
decoder_input_data = np.zeros((X_train.shape[0], max_length), dtype=int)  # (15608, 30)

# Set the first token as the start token and shift the y_train for decoder input
decoder_input_data[:, 0] = start_token_id  # Start token
decoder_input_data[:, 1:] = y_train[:, :-1]  # Shifted input for decoder

# Prepare the decoder target data (shifted y_train)
decoder_target_data = y_train[:, 1:]  # Now it has shape (15608, 29)

# Pad target data to match (15608, 30)
decoder_target_data = np.pad(decoder_target_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)  # Now (15608, 30)

# Debug shapes
print(f"Shape of encoder input data: {X_train.shape}")  # Should be (15608, 30)
print(f"Shape of decoder input data: {decoder_input_data.shape}")  # Should be (



Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18            │ (None, 30)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_18 (Embedding)  │ (None, 30, 512)        │      5,744,128 │ input_layer_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_19            │ (None, 30)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_14          │ [(None, 30, 256),      │        656,384 │ embedding_18[0][0]     │
│ (Bidirectional)           │ (None, 128), (None,    │                │                        │
│                           │ 128), (None, 128),     │                │                        │
│                           │ (None, 128)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_19 (Embedding)  │ (None, 30, 512)        │      6,145,536 │ input_layer_19[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_18            │ (None, 256)            │              0 │ bidirectional_14[0][1… │
│ (Concatenate)             │                        │                │ bidirectional_14[0][3] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_19            │ (None, 256)            │              0 │ bidirectional_14[0][2… │
│ (Concatenate)             │                        │                │ bidirectional_14[0][4] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_24 (LSTM)            │ [(None, 30, 256),      │        787,456 │ embedding_19[0][0],    │
│                           │ (None, 256), (None,    │                │ concatenate_18[0][0],  │
│                           │ 256)]                  │                │ concatenate_19[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_9     │ (None, 30, 256)        │          1,024 │ lstm_24[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_19 (Dropout)      │ (None, 30, 256)        │              0 │ batch_normalization_9… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_9        │ (None, 30, 12003)      │      3,084,771 │ dropout_19[0][0]       │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 16,419,299 (62.63 MB)

 Trainable params: 16,418,787 (62.63 MB)

 Non-trainable params: 512 (2.00 KB)

Shape of encoder input data: (15608, 30)
Shape of decoder input data: (15608, 30)


In [ ]:
batch_size = 32
epochs = 40

# Model fitting
history = model.fit(
    [X_train, decoder_input_data],
    decoder_target_data,
    validation_data=([X_val, decoder_input_val], decoder_target_val),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[ model_checkpoint, reduce_lr]
)

Epoch 1/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 40s 73ms/step - accuracy: 0.6097 - loss: 3.6719 - val_accuracy: 0.6647 - val_loss: 2.4167 - learning_rate: 0.0010
Epoch 2/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 30s 61ms/step - accuracy: 0.6863 - loss: 2.0571 - val_accuracy: 0.6558 - val_loss: 2.1730 - learning_rate: 0.0010
Epoch 3/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 60ms/step - accuracy: 0.7261 - loss: 1.5670 - val_accuracy: 0.6580 - val_loss: 2.1725 - learning_rate: 0.0010
Epoch 4/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 28s 57ms/step - accuracy: 0.7539 - loss: 1.2537 - val_accuracy: 0.6568 - val_loss: 2.3004 - learning_rate: 0.0010
Epoch 5/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.7847 - loss: 0.9875 - val_accuracy: 0.6559 - val_loss: 2.4182 - learning_rate: 0.0010
Epoch 6/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.8194 - loss: 0.7622 - val_accuracy: 0.6569 - val_loss: 2.5519 - learning_rate: 0.0010
Epoch 7/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 57ms/step - accuracy: 0.8562 - l

In [ ]:
model.save('model3_96-12.h5.keras')

In [ ]:
model.save('encoded_decoded_model3_96-12_65-3.4.h5.keras')
model.save_weights('encoded_decoded_model3_96-12_65.weights.h5')
model.save_own_variables('encoded_decoded_model3_96-12_65-variables.h5')


In [ ]:
model=tf.keras.models.load_model('model2.h5.keras')

In [ ]:
def prepare_test_case(sentence, max_length_eng):
    # Tokenize and pad the sentence
    seq = eng_tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=max_length_eng, padding='post')
    return padded

def translate_sentence(sentence):
    # Prepare the test case
    test_input = prepare_test_case(sentence, 30)

    # Generate an initial input for the decoder
    decoder_input = np.zeros((1, 30))  # (batch_size, max_length_urd)
    decoder_input[0, 0] = urd_tokenizer.word_index['<start>']  # Assuming '<start>' token is defined

    # Iterate for each time step of the decoder
    for i in range(1, 30):
        output = model.predict([test_input, decoder_input])
        sampled_token_index = np.argmax(output[0, i - 1, :])  # Sample the most likely token
        decoder_input[0, i] = sampled_token_index

        # Break if the model predicts the end token
        if sampled_token_index == urd_tokenizer.word_index['<end>']:  # Assuming '<end>' token is defined
            break

    # Convert the predicted sequence back to text
    translated_sentence = urd_tokenizer.sequences_to_texts([decoder_input[0]])[0]
    return translated_sentence

# Example usage
test_sentence = "where are you going"  # Replace with an actual English sentence
translated_output = translate_sentence(test_sentence)
print(f'Translated Sentence: {translated_output}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Translated Sentence: <start> کہاں ہیں <end>


#2nd architure of Encoder Decoder

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Concatenate, Dense, TimeDistributed
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters
embedding_dim = 512  # Dimension of the embedding layer
max_length = 30  # Maximum length for input and output sequences

# Encoder
encoder_input = Input(shape=(max_length,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=embedding_dim)(encoder_input)
encoder_lstm = Bidirectional(LSTM(256, return_state=True, return_sequences=True))
encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
encoder_lstm = Bidirectional(LSTM(128, return_state=True, return_sequences=True))
encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

encoder_output = Dropout(0.6)(encoder_output)
encoder_output=BatchNormalization()(encoder_output)

# Combine the states from the forward and backward LSTMs
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Decoder
decoder_input = Input(shape=(max_length,))
decoder_embedding = Embedding(input_dim=urd_vocab_size, output_dim=embedding_dim)(decoder_input)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])
# decoder_lstm = LSTM(512, return_sequences=True, return_state=True)
# decoder_output, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

decoder_output = BatchNormalization()(decoder_output)

decoder_output = Dropout(0.4)(decoder_output)
decoder_dense = TimeDistributed(Dense(urd_vocab_size, activation='softmax'))
decoder_output = decoder_dense(decoder_output)

# Define the model
model = Model([encoder_input, decoder_input], decoder_output)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
start_token_id = 1  # Assume <start> token is 1

# Initialize decoder input data
decoder_input_data = np.zeros((X_train.shape[0], max_length), dtype=int)  # (15608, 30)

# Set the first token as the start token and shift the y_train for decoder input
decoder_input_data[:, 0] = start_token_id  # Start token
decoder_input_data[:, 1:] = y_train[:, :-1]  # Shifted input for decoder

# Prepare the decoder target data (shifted y_train)
decoder_target_data = y_train[:, 1:]  # Now it has shape (15608, 29)

# Pad target data to match (15608, 30)
decoder_target_data = np.pad(decoder_target_data, ((0, 0), (0, 1)), mode='constant', constant_values=0)  # Now (15608, 30)

# Debug shapes
print(f"Shape of encoder input data: {X_train.shape}")  # Should be (15608, 30)
print(f"Shape of decoder input data: {decoder_input_data.shape}")  # Should be (



Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, 30)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_20 (Embedding)  │ (None, 30, 512)        │      5,744,128 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_21            │ (None, 30)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_16          │ [(None, 30, 256),      │        656,384 │ embedding_20[0][0]     │
│ (Bidirectional)           │ (None, 128), (None,    │                │                        │
│                           │ 128), (None, 128),     │                │                        │
│                           │ (None, 128)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_21 (Embedding)  │ (None, 30, 512)        │      6,145,536 │ input_layer_21[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_20            │ (None, 256)            │              0 │ bidirectional_16[0][1… │
│ (Concatenate)             │                        │                │ bidirectional_16[0][3] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_21            │ (None, 256)            │              0 │ bidirectional_16[0][2… │
│ (Concatenate)             │                        │                │ bidirectional_16[0][4] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_27 (LSTM)            │ [(None, 30, 256),      │        787,456 │ embedding_21[0][0],    │
│                           │ (None, 256), (None,    │                │ concatenate_20[0][0],  │
│                           │ 256)]                  │                │ concatenate_21[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_11    │ (None, 30, 256)        │          1,024 │ lstm_27[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_21 (Dropout)      │ (None, 30, 256)        │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed_10       │ (None, 30, 12003)      │      3,084,771 │ dropout_21[0][0]       │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 16,419,299 (62.63 MB)

 Trainable params: 16,418,787 (62.63 MB)

 Non-trainable params: 512 (2.00 KB)

Shape of encoder input data: (15608, 30)
Shape of decoder input data: (15608, 30)


In [ ]:
batch_size = 32
epochs = 40

# Model fitting
history = model.fit(
    [X_train, decoder_input_data],
    decoder_target_data,
    validation_data=([X_val, decoder_input_val], decoder_target_val),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[ model_checkpoint, reduce_lr]
)

Epoch 1/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 40s 73ms/step - accuracy: 0.6042 - loss: 3.7189 - val_accuracy: 0.6652 - val_loss: 2.2515 - learning_rate: 0.0010
Epoch 2/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 30s 61ms/step - accuracy: 0.6745 - loss: 2.2072 - val_accuracy: 0.6564 - val_loss: 2.1936 - learning_rate: 0.0010
Epoch 3/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 39s 58ms/step - accuracy: 0.7072 - loss: 1.7678 - val_accuracy: 0.6540 - val_loss: 2.2424 - learning_rate: 0.0010
Epoch 4/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 57ms/step - accuracy: 0.7250 - loss: 1.5503 - val_accuracy: 0.6556 - val_loss: 2.3244 - learning_rate: 0.0010
Epoch 5/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.7457 - loss: 1.3330 - val_accuracy: 0.6568 - val_loss: 2.2832 - learning_rate: 0.0010
Epoch 6/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.7719 - loss: 1.1017 - val_accuracy: 0.6573 - val_loss: 2.4074 - learning_rate: 5.0000e-04
Epoch 7/40
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 58ms/step - accuracy: 0.7903

#Bidirectional LSTM

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


#useing the tokeinzer on english texts
eng_tokenizer=Tokenizer()
eng_tokenizer.fit_on_texts(df['SENTENCES '])
eng_vocab_size=len(eng_tokenizer.word_index)+1
print('English Vocabulary Size: %d' % eng_vocab_size)
eng_tokenizer.texts_to_sequences(df['SENTENCES '])
padded_eng_sentences=pad_sequences(eng_tokenizer.texts_to_sequences(df['SENTENCES ']),padding='post')


urd_tokenizer=Tokenizer()
urd_tokenizer.fit_on_texts(df['MEANING'])
urd_vocab_size=len(urd_tokenizer.word_index)+1


print('Urdu Vocabulary Size: %d' % urd_vocab_size)
encoded_urdu_sentences=urd_tokenizer.texts_to_sequences(df['MEANING'])
padded_urdu_sentences=pad_sequences(encoded_urdu_sentences,padding='post')

X=padded_eng_sentences
y=padded_urdu_sentences

# Split the dataset into train and test sets (80% train, 20% test)
# X_train, X_val, y_train, y_val = train_test_split(padded_eng_sentences, padded_urdu_sentences, test_size=0.1, random_state=42)

X_train, X_temp, y_train, y_temp = train_test_split(padded_eng_sentences, padded_urdu_sentences, test_size=0.2, random_state=42)

# Now, split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
max_input_length = 31  # or 30 based on your target
X_train_padded = pad_sequences(X_train, maxlen=max_input_length, padding='post')
X_val_padded = pad_sequences(X_val, maxlen=max_input_length, padding='post')

# If you pad inputs, adjust your target shapes accordingly:
y_train_padded = pad_sequences(y_train, maxlen=max_input_length, padding='post')
y_val_padded = pad_sequences(y_val, maxlen=max_input_length, padding='post')


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional,Dropout,BatchNormalization,TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_input_length = 31  # or 30 based on your target
X_train_padded = pad_sequences(X_train, maxlen=max_input_length, padding='post')
X_val_padded = pad_sequences(X_val, maxlen=max_input_length, padding='post')
y_train_padded = pad_sequences(y_train, maxlen=max_input_length, padding='post')
y_val_padded = pad_sequences(y_val, maxlen=max_input_length, padding='post')
model4 = Sequential([
    Embedding(input_dim=eng_vocab_size, output_dim=256, input_length=30),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(256, kernel_regularizer=tf.keras.regularizers.l2(0.01), return_sequences=True)),
    BatchNormalization(),
    TimeDistributed(Dense(urd_vocab_size, activation='softmax'))
])
# callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)]
model4.compile(optimizer=Adam(learning_rate=0.0005),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model4.summary()
history4 = model4.fit(X_train_padded, y_train_padded, epochs=30, batch_size=32, validation_data=(X_val_padded, y_val_padded))


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 46s 75ms/step - accuracy: 0.6317 - loss: 6.9433 - val_accuracy: 0.6670 - val_loss: 2.3949
Epoch 2/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 76ms/step - accuracy: 0.6638 - loss: 2.3895 - val_accuracy: 0.6698 - val_loss: 2.3362
Epoch 3/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 42s 78ms/step - accuracy: 0.6735 - loss: 2.2425 - val_accuracy: 0.6737 - val_loss: 2.2590
Epoch 4/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 40s 76ms/step - accuracy: 0.6766 - loss: 2.1536 - val_accuracy: 0.6763 - val_loss: 2.2252
Epoch 5/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 0.6811 - loss: 2.0680 - val_accuracy: 0.6803 - val_loss: 2.1594
Epoch 6/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 0.6861 - loss: 1.9898 - val_accuracy: 0.6826 - val_loss: 2.1338
Epoch 7/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 0.6911 - loss: 1.9178 - val_accuracy: 0.6852 - val_loss: 2.1180
Epoch 8/30
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 0.6938 - loss: 1.8630 - 

In [ ]:
model4.save('model30ep80-85_70-2.2.h5.keras')

In [ ]:
model4_1=tf.keras.models.load_model('/content/model30ep80-85_70-2.2.h5.keras')

In [ ]:
model4_1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 31, 256)             │       2,871,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ (None, 31, 512)             │       1,050,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 31, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_10 (Bidirectional)     │ (None, 31, 512)             │       1,574,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 31, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_11 (Bidirectional)     │ (None, 31, 512)             │       1,574,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 31, 512)             │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 31, 11099)           │       5,693,787 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,302,227 (146.11 MB)

 Trainable params: 12,767,067 (48.70 MB)

 Non-trainable params: 1,024 (4.00 KB)

 Optimizer params: 25,534,136 (97.41 MB)

In [ ]:
X_test_padded = pad_sequences(X_test, maxlen=max_input_length, padding='post')
y_test_padded = pad_sequences(y_test, maxlen=max_input_length, padding='post')


In [ ]:
model4_1.evaluate(X_test_padded,y_test_padded)

61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.7090 - loss: 2.2647


[2.2662839889526367, 0.708322286605835]

In [ ]:
sentence='iam going to communicate with my parents'
eng_tokenizer.texts_to_sequences([sentence])
pad_eng_sentence=pad_sequences(eng_tokenizer.texts_to_sequences([sentence]),maxlen=30,padding='post')
predictions=model4_1.predict(pad_eng_sentence)
predicted_seq=np.argmax(predictions,axis=-1)
# predicted_seq
# Create a reverse mapping from index to word for Urdu
urd_index_word = {v: k for k, v in urd_tokenizer.word_index.items()}

# Convert predicted sequences to words
predicted_urdu_translation = []
for word_index in predicted_seq[0]:  # Assuming a single input sentence
    if word_index != 0:  # Skip padding
        predicted_urdu_translation.append(urd_index_word.get(word_index, ''))

# Join the predicted words to form the translated sentence
translated_sentence = ' '.join(predicted_urdu_translation)
print("Predicted Urdu Translation:", translated_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Urdu Translation: ‘‘جب کے بات بات بات


In [ ]:
model4_1.fit(X_train_padded, y_train_padded, epochs=10, batch_size=32, validation_data=(X_val_padded, y_val_padded))

Epoch 1/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 42s 77ms/step - accuracy: 0.8068 - loss: 0.8391 - val_accuracy: 0.7021 - val_loss: 2.3012
Epoch 2/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 36s 74ms/step - accuracy: 0.8085 - loss: 0.8357 - val_accuracy: 0.7046 - val_loss: 2.2942
Epoch 3/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 42s 76ms/step - accuracy: 0.8141 - loss: 0.7979 - val_accuracy: 0.7030 - val_loss: 2.3187
Epoch 4/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 76ms/step - accuracy: 0.8195 - loss: 0.7748 - val_accuracy: 0.7052 - val_loss: 2.3287
Epoch 5/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 0.8204 - loss: 0.7641 - val_accuracy: 0.7029 - val_loss: 2.3534
Epoch 6/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 75ms/step - accuracy: 0.8271 - loss: 0.7363 - val_accuracy: 0.7026 - val_loss: 2.3585
Epoch 7/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8294 - loss: 0.7224 - val_accuracy: 0.7049 - val_loss: 2.3825
Epoch 8/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8323 - loss: 0.7131 - 

In [ ]:
model4_1.save('model40ep83-67_70-2.4.h5.keras')

In [ ]:
sentence='iam going to communicate with my parents'
eng_tokenizer.texts_to_sequences([sentence])
pad_eng_sentence=pad_sequences(eng_tokenizer.texts_to_sequences([sentence]),maxlen=30,padding='post')
predictions=model4_1.predict(pad_eng_sentence)
predicted_seq=np.argmax(predictions,axis=-1)
# predicted_seq
# Create a reverse mapping from index to word for Urdu
urd_index_word = {v: k for k, v in urd_tokenizer.word_index.items()}

# Convert predicted sequences to words
predicted_urdu_translation = []
for word_index in predicted_seq[0]:  # Assuming a single input sentence
    if word_index != 0:  # Skip padding
        predicted_urdu_translation.append(urd_index_word.get(word_index, ''))

# Join the predicted words to form the translated sentence
translated_sentence = ' '.join(predicted_urdu_translation)
print("Predicted Urdu Translation:", translated_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted Urdu Translation: ‘‘جب کے والدین بات بات


In [ ]:

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
model4_1.fit(X_train_padded, y_train_padded, epochs=10, batch_size=32, validation_data=(X_val_padded, y_val_padded),callbacks=[early_stopping,reduce_lr])

Epoch 1/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 76ms/step - accuracy: 0.8409 - loss: 0.6669 - val_accuracy: 0.7034 - val_loss: 2.4514 - learning_rate: 5.0000e-04
Epoch 2/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 75ms/step - accuracy: 0.8447 - loss: 0.6482 - val_accuracy: 0.7068 - val_loss: 2.4549 - learning_rate: 5.0000e-04
Epoch 3/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 42s 76ms/step - accuracy: 0.8484 - loss: 0.6315 - val_accuracy: 0.7053 - val_loss: 2.4657 - learning_rate: 5.0000e-04
Epoch 4/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 40s 75ms/step - accuracy: 0.8495 - loss: 0.6260 - val_accuracy: 0.7050 - val_loss: 2.4999 - learning_rate: 5.0000e-04
Epoch 5/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 75ms/step - accuracy: 0.8660 - loss: 0.5518 - val_accuracy: 0.7086 - val_loss: 2.4432 - learning_rate: 5.0000e-05
Epoch 6/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 76ms/step - accuracy: 0.8790 - loss: 0.4815 - val_accuracy: 0.7095 - val_loss: 2.4454 - learning_rate: 5.0000e-05
Epoch 7/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 37s 75ms/ste

In [ ]:
model4_1.save('model40ep89-41_70-2.4.h5.keras')

In [ ]:
model4_1.evaluate(X_test_padded,y_test_padded)

61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.7130 - loss: 2.4549


[2.458451271057129, 0.7126520872116089]

#Best model so far

In [ ]:
# sentence='iam going to communicate with my parents'
sentence='why how'
eng_tokenizer.texts_to_sequences([sentence])
pad_eng_sentence=pad_sequences(eng_tokenizer.texts_to_sequences([sentence]),maxlen=30,padding='post')
predictions=model4_1.predict(pad_eng_sentence)
predicted_seq=np.argmax(predictions,axis=-1)
# predicted_seq
# Create a reverse mapping from index to word for Urdu
urd_index_word = {v: k for k, v in urd_tokenizer.word_index.items()}

# Convert predicted sequences to words
predicted_urdu_translation = []
for word_index in predicted_seq[0]:  # Assuming a single input sentence
    if word_index != 0:  # Skip padding
        predicted_urdu_translation.append(urd_index_word.get(word_index, ''))

# Join the predicted words to form the translated sentence
translated_sentence = ' '.join(predicted_urdu_translation)
print("Predicted Urdu Translation:", translated_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Urdu Translation: کیوں


In [ ]:
model3=tf.keras.models.load_model('/content/model40ep89-41_70-2.4.h5.keras')

In [ ]:
model3.fit(X_train_padded, y_train_padded, epochs=10, batch_size=32, validation_data=(X_val_padded, y_val_padded))

Epoch 1/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.8805 - loss: 0.4777 - val_accuracy: 0.7093 - val_loss: 2.4408
Epoch 2/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 36s 74ms/step - accuracy: 0.8809 - loss: 0.4741 - val_accuracy: 0.7095 - val_loss: 2.4388
Epoch 3/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8818 - loss: 0.4703 - val_accuracy: 0.7097 - val_loss: 2.4376
Epoch 4/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 36s 75ms/step - accuracy: 0.8838 - loss: 0.4635 - val_accuracy: 0.7095 - val_loss: 2.4394
Epoch 5/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8850 - loss: 0.4551 - val_accuracy: 0.7099 - val_loss: 2.4358
Epoch 6/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8854 - loss: 0.4538 - val_accuracy: 0.7099 - val_loss: 2.4392
Epoch 7/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8841 - loss: 0.4594 - val_accuracy: 0.7101 - val_loss: 2.4420
Epoch 8/10
488/488 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.8865 - loss: 0.4481 - 

In [ ]:
sentence='good morning'
eng_tokenizer.texts_to_sequences([sentence])
pad_eng_sentence=pad_sequences(eng_tokenizer.texts_to_sequences([sentence]),maxlen=30,padding='post')
predictions=model3.predict(pad_eng_sentence)
predicted_seq=np.argmax(predictions,axis=-1)
# predicted_seq
# Create a reverse mapping from index to word for Urdu
urd_index_word = {v: k for k, v in urd_tokenizer.word_index.items()}

# Convert predicted sequences to words
predicted_urdu_translation = []
for word_index in predicted_seq[0]:  # Assuming a single input sentence
    if word_index != 0:  # Skip padding
        predicted_urdu_translation.append(urd_index_word.get(word_index, ''))

# Join the predicted words to form the translated sentence
translated_sentence = ' '.join(predicted_urdu_translation)
print("Predicted Urdu Translation:", translated_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted Urdu Translation: صبح بخير۔


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer # Import WordNetLemmatizer
from nltk.tokenize import word_tokenize # Import word_tokenize

# Download necessary NLTK data for lemmatization
nltk.download('punkt')
nltk.download('wordnet')

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Join the lemmatized words back into a sentence
    return ' '.join(lemmatized_words)

def translate_sentence(sentence, eng_tokenizer, urd_tokenizer, model, maxlen=30):
    # Lemmatize the sentence
    sentence.lower()
    sentence = lemmatize_sentence(sentence)


    # Tokenize and pad the input English sentence
    pad_eng_sentence = pad_sequences(eng_tokenizer.texts_to_sequences([sentence]), maxlen=maxlen, padding='post')

    # Make predictions using the model
    predictions = model.predict(pad_eng_sentence)

    # Get the predicted word indices
    predicted_seq = np.argmax(predictions, axis=-1)

    # Create a reverse mapping from index to word for Urdu
    urd_index_word = {v: k for k, v in urd_tokenizer.word_index.items()}

    # Convert predicted sequences to words
    predicted_urdu_translation = []
    for word_index in predicted_seq[0]:  # Assuming a single input sentence
        if word_index != 0:  # Skip padding
            predicted_urdu_translation.append(urd_index_word.get(word_index, ''))

    # Join the predicted words to form the translated sentence
    translated_sentence = ' '.join(predicted_urdu_translation)

    return translated_sentence
# Example usage
test_sentence = 'good morning'
translated = translate_sentence(test_sentence, eng_tokenizer, urd_tokenizer, model3)
print('english sentence:',test_sentence,' ',"Predicted Urdu Translation:", translated)

test_sentence = 'how are you'
translated = translate_sentence(test_sentence, eng_tokenizer, urd_tokenizer, model3)
print('english sentence:',test_sentence,' ',"Predicted Urdu Translation:", translated)

test_sentence = 'car'
translated = translate_sentence(test_sentence, eng_tokenizer, urd_tokenizer, model3)
print('english sentence:',test_sentence,' ',"Predicted Urdu Translation:", translated)

test_sentence = 'i am good'
translated = translate_sentence(test_sentence, eng_tokenizer, urd_tokenizer, model3)
print('english sentence:',test_sentence,' ',"Predicted Urdu Translation:", translated)

test_sentence = 'what do you want'
translated = translate_sentence(test_sentence, eng_tokenizer, urd_tokenizer, model3)
print('english sentence:',test_sentence,' ',"Predicted Urdu Translation:", translated)

test_sentence = 'he sit on sofa'
translated = translate_sentence(test_sentence, eng_tokenizer, urd_tokenizer, model3)
print('english sentence:',test_sentence,' ',"Predicted Urdu Translation:", translated)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
english sentence: good morning   Predicted Urdu Translation: صبح بخير۔
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
english sentence: how are you   Predicted Urdu Translation: آپ کیسی ہیں
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
english sentence: car   Predicted Urdu Translation: گاڑی
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
english sentence: i am good   Predicted Urdu Translation: مجھے اچھا ہوتا ہوں۔
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
english sentence: what do you want   Predicted Urdu Translation: آپ کیا چاہتےہیں ہیں ہیںَ
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
english sentence: he sit on sofa   Predicted Urdu Translation: ہم نے پر پر بنا ہیں۔


#END of REFININF LSTM